# Dota 2 数据集清洗

基于kaggle数据集Dota 2 Matches（https://www.kaggle.com/datasets/devinanzelmo/dota-2-matches ）
实现逐表清洗：读取→映射→清洗→保存。  


> 路径约定：原始 CSV 文件应放在 `DATA_DIR` 指定的路径下（默认为 `/workspace/Dota2data`）；清洗输出文件将保存在 `OUT_DIR`（默认为 `/workspace/cleaned`）中。  
> 清洗内容：缺失值填充、重复值去除、异常值处理、字段类型转换、整合映射表（英雄/物品/补丁/区域）、数据一致性检查等。


## 依赖与路径配置

导入所需的库，并指定原始数据和清洗输出的目录路径。然后创建输出目录（如果尚不存在），以确保清洗后的文件可以正确保存。最后，打印数据目录和输出目录的绝对路径以确认设置是否正确。


In [1]:
# 依赖与路径配置
import os
from pathlib import Path
import pandas as pd
import numpy as np

DATA_DIR = Path('Dota2data')   # 原始 CSV 数据目录
OUT_DIR  = Path('cleaned')     # 清洗后的 CSV 输出目录
OUT_DIR.mkdir(exist_ok=True)             # 创建输出目录（如不存在）

# 输出数据目录路径，确认当前有效
print("数据目录:", DATA_DIR.resolve())
print("清洗输出目录:", OUT_DIR.resolve())


数据目录: D:\Code\Dota2datacleaning\Dota2data
清洗输出目录: D:\Code\Dota2datacleaning\cleaned


## 通用工具函数

封装了一些重复使用的工具函数，用于文件读写和数据处理：

- **`read_csv_if_exists`**：尝试读取指定路径的 CSV 文件（如果路径不存在则在 `DATA_DIR` 下查找），返回相应的 DataFrame；如文件缺失或读取错误，则打印警告/错误信息并返回 None。  
- **`save_df`**：将 DataFrame 保存为 CSV 文件（文件名会加 `_clean` 后缀），保存在 `OUT_DIR` 目录下，并打印保存成功的信息。  
- **`boolify01`**：将系列中的真/假表示（如 0/1、字符串 "True"/"False" 等）统一转换为布尔值。  
- **`ensure_datetime`**：将系列转换为日期时间格式；如果提供了时间单位参数（如秒 `'s'`），则按该单位解析，否则自动解析。  
- **`clamp`**：将数值限制在指定的上下限之间，低于下限的设为下限，超过上限的设为上限。  
- **`add_team_from_slot`**：根据 `player_slot` 列添加对应的阵营列（天辉 Radiant 或夜魇 Dire），以方便按阵营分组统计。


In [2]:
# 通用工具函数
def read_csv_if_exists(name_or_path, nrows=None):
    # 尝试读取 CSV 文件（先按提供路径，否则从 DATA_DIR 下查找）
    p = Path(name_or_path)
    if not p.exists():
        p = DATA_DIR / str(name_or_path)
    if not p.exists():
        print(f"[WARN] 未找到文件：{name_or_path}")
        return None
    try:
        df = pd.read_csv(p, nrows=nrows)
        print(f"[READ] {p.name}: shape={df.shape}")
        return df
    except Exception as e:
        print(f"[ERROR] 读取失败 {p}: {e}")
        return None

def save_df(df, fname):
    # 保存 DataFrame 为 CSV 文件（添加 '_clean' 后缀），返回输出文件路径
    out_path = OUT_DIR / f"{fname}_clean.csv"
    df.to_csv(out_path, index=False)
    print(f"[SAVE] {out_path} -> shape={df.shape}")
    return out_path

def boolify01(series: pd.Series) -> pd.Series:
    # 将序列中的 True/False/0/1 等值转换为布尔类型
    return series.astype(str).str.strip().replace({'True': '1', 'False': '0'}).astype(float).fillna(0).astype(int).astype(bool)

def ensure_datetime(series: pd.Series, unit=None):
    # 将序列转换为日期时间（若提供 unit 则按该时间单位解析）
    try:
        return pd.to_datetime(series, unit=unit, errors='coerce') if unit else pd.to_datetime(series, errors='coerce')
    except Exception:
        # 兜底：无法解析则返回全 NaT 序列
        return pd.to_datetime(pd.Series([pd.NaT] * len(series)))

def clamp(series: pd.Series, lo=None, hi=None):
    # 将数值序列限定在区间 [lo, hi] 范围内
    s = pd.to_numeric(series, errors='coerce')
    if lo is not None:
        s = np.maximum(s, lo)
    if hi is not None:
        s = np.minimum(s, hi)
    return s

def add_team_from_slot(df: pd.DataFrame, slot_col='player_slot'):
    # 基于 slot_col 列值添加阵营列（Radiant 或 Dire）
    if slot_col in df.columns:
        slot = pd.to_numeric(df[slot_col], errors='coerce').fillna(0).astype(int)
        df['team'] = np.where(slot < 128, 'Radiant', 'Dire')
    return df


## 载入映射表并统一列名

读取各种辅助映射表（维表）并进行必要的列名规范化，为后续清洗提供参照数据：

- 加载 **cluster_regions.csv**（集群 ID 到区域名称的映射表）。
- 加载 **patch_dates.csv** 并统一列名（例如将 `patch_date` 重命名为 `date`，`name` 重命名为 `patch`），并将日期字段转换为 datetime 时间戳。
- 加载 **hero_names.csv**，规范英雄名称列名（如存在 `localized_name` 列则改为统一的 `hero_name`），并如有需要生成匹配主数据的英雄 ID 键（`hero_id_key`）；最终得到英雄名称映射表 **hero_map**。
- 分别加载 **item_ids.csv** 与 **ability_ids.csv**，提取物品和技能的 ID-名称映射表（去除重复条目）。


In [3]:
# 载入映射表并统一列名
cluster_df = read_csv_if_exists('cluster_regions.csv')  # 区域映射表（cluster -> region）
patch_df   = read_csv_if_exists('patch_dates.csv')     # 补丁版本映射表
hero_df    = read_csv_if_exists('hero_names.csv')      # 英雄名称映射表
item_df    = read_csv_if_exists('item_ids.csv')        # 物品名称映射表
ability_df = read_csv_if_exists('ability_ids.csv')     # 技能名称映射表

# 补丁表列名调整：patch_date -> date, name -> patch
if patch_df is not None:
    patch_df = patch_df.rename(columns={'patch_date':'date', 'name':'patch'})
    patch_df['date'] = pd.to_datetime(patch_df['date'], errors='coerce')
    patch_df['date'] = patch_df['date'].dt.tz_localize(None)
    patch_df['patch'] = patch_df['patch'].astype(str)

# 英雄名表处理：统一名称列名，生成 hero_id_key 键
hero_map = None
if hero_df is not None:
    h = hero_df.copy()
    if 'hero_name' not in h.columns and 'localized_name' in h.columns:
        h = h.rename(columns={'localized_name': 'hero_name'})
    if 'hero_id' in h.columns:
        h['hero_id_key'] = pd.to_numeric(h['hero_id'], errors='coerce')
        hero_map = h[['hero_id_key','hero_name']].drop_duplicates('hero_id_key')
        print(f"[MAP] hero_map rows = {len(hero_map)}")

# 物品/技能映射表去重
if item_df is not None:
    item_df = item_df[['item_id','item_name']].drop_duplicates('item_id')
if ability_df is not None:
    ability_df = ability_df[['ability_id','ability_name']].drop_duplicates('ability_id')


[READ] cluster_regions.csv: shape=(53, 2)
[READ] patch_dates.csv: shape=(19, 2)
[READ] hero_names.csv: shape=(112, 3)
[READ] item_ids.csv: shape=(189, 2)
[READ] ability_ids.csv: shape=(688, 2)
[MAP] hero_map rows = 112


## 定义各表清洗函数

针对每个数据表定义清洗函数，封装具体的清理操作：

- **clean_matches**：清洗比赛主表。转换胜负标志为布尔值，生成比赛开始时间列，标记超长时长的比赛；合并地区（cluster -> region）和版本补丁映射；最后去除重复的比赛记录。
- **clean_players**：清洗玩家数据。填充缺失值为 0，去除重复的 (match_id, player_slot) 记录并添加阵营列；截断异常的大击杀数值并标记；合并英雄名称映射表。
- **clean_player_time**：清洗时间序列数据。移除无效的时间点记录（如负值），去除重复的 (match_id, times) 记录。
- **clean_teamfights**：清洗团队战斗事件数据。移除开始/结束时间异常的记录，去除重复的 (match_id, start, end) 记录。
- **clean_teamfights_players**：清洗团队战斗玩家数据。填充缺失数值，去除重复记录并添加阵营列。
- **clean_objectives**：清洗目标事件数据。统一列名，移除时间异常的记录，补充阵营列，去除重复记录，如无 `type` 列则填入默认值。
- **clean_chat**：清洗聊天记录数据。统一列名，移除时间异常的记录，并添加阵营列。
- **clean_test_labels**：清洗测试集比赛标签。转换胜负标签为布尔值，去除重复的 match_id 记录。
- **clean_test_player**：清洗测试集玩家数据。删除缺失关键字段的记录，去除重复的 (match_id, player_slot) 记录；合并英雄名称映射表并添加阵营列。
- **clean_player_ratings**：清洗玩家评分数据。裁剪非法的胜场数（wins > matches）并修正为合理值，将评分指标限定为非负，去除重复的 account_id 记录。
- **clean_match_outcomes**：清洗比赛结果汇总数据。确保关键数值列为整数类型，去除重复的 match_id 记录。
- **clean_purchase_log**：清洗物品购买日志。移除异常的时间记录（过早或负值），计算每条记录发生的分钟数；合并物品名称映射表并添加阵营列。
- **clean_ability_upgrades**：清洗技能升级日志。统一列名，限制升级等级不低于1，移除异常的时间记录；合并技能名称映射表并添加阵营列。


In [4]:
# 各表清洗函数
def clean_matches(df: pd.DataFrame, cluster_df=None, patch_df=None):
    df = df.copy()  # 拷贝数据，避免修改原始 DataFrame
    # 胜负标签 0/1 转换为布尔值 True/False
    if 'radiant_win' in df.columns:
        df['radiant_win'] = boolify01(df['radiant_win'])
    # 时间戳转换为 datetime（单位：秒）
    if 'start_time' in df.columns:
        df['start_time_dt'] = ensure_datetime(df['start_time'], unit='s')
    # 标记超长比赛（持续时间 > 2 小时）
    if 'duration' in df.columns:
        df['long_match'] = pd.to_numeric(df['duration'], errors='coerce') > 7200
    # 合并区域映射（cluster -> region）
    if cluster_df is not None and 'cluster' in df.columns and 'cluster' in cluster_df.columns:
        df = df.merge(cluster_df.drop_duplicates('cluster'), on='cluster', how='left')
        miss = df['region'].isna().sum() if 'region' in df.columns else 0
        if miss > 0:
            print(f"[matches] 有 {miss} 场比赛 cluster 未映射到 region")
    # 合并补丁版本（根据开始时间匹配所属补丁）
    if patch_df is not None and 'start_time_dt' in df.columns and 'date' in patch_df.columns:
        pdf = patch_df.sort_values('date')[['patch','date']].rename(columns={'date':'patch_date'})
        try:
            df = pd.merge_asof(df.sort_values('start_time_dt'), pdf.sort_values('patch_date'),
                               left_on='start_time_dt', right_on='patch_date', direction='backward')
        except Exception as e:
            print(f"[WARN] merge_asof 失败：{e}")
    # 主键唯一性检查（match_id 不应重复）
    if 'match_id' in df.columns:
        before_len = len(df)
        df = df.drop_duplicates('match_id', keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[matches] 警告：match_id 出现重复 {dup} 条，已保留第一条")
    return df

def clean_players(df: pd.DataFrame, hero_map=None):
    df = df.copy()  
    # 填充数值列中的缺失值为 0
    num_cols = df.select_dtypes(include=[np.number]).columns
    na_total = df[num_cols].isna().sum().sum()
    df[num_cols] = df[num_cols].fillna(0)
    if na_total > 0:
        print(f"[players] 填充缺失值：{na_total} 个 -> 0")
    # 复合键唯一性检查（match_id, player_slot）
    keys = [c for c in ['match_id','player_slot'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[players] 删除重复 (match_id, player_slot)：{dup} 条")
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    # 限制异常击杀数值并标记
    if 'kills' in df.columns:
        mask = pd.to_numeric(df['kills'], errors='coerce') > 100
        df['kills_outlier'] = False
        if mask.any():
            df.loc[mask, 'kills'] = 100
            df.loc[mask, 'kills_outlier'] = True
            print(f"[players] 发现异常击杀 {mask.sum()} 条（kills>100），已裁剪至100 并标记 kills_outlier")
    # 合并英雄名称映射
    if hero_map is not None and 'hero_id' in df.columns and 'hero_id_key' in hero_map.columns:
        df['hero_id'] = pd.to_numeric(df['hero_id'], errors='coerce')
        df = df.merge(hero_map[['hero_id_key','hero_name']], left_on='hero_id', right_on='hero_id_key', how='left')
        miss = df['hero_name'].isna().sum()
        if miss > 0:
            print(f"[players] 警告：有 {miss} 条 hero_id 未映射到名称（可能是新英雄/映射表需更新）")
        df.drop(columns=['hero_id_key'], inplace=True, errors='ignore')
    return df

def clean_player_time(df: pd.DataFrame):
    df = df.copy()  
    # 移除无效时间（负值）记录
    if 'times' in df.columns:
        before_len = len(df)
        df = df[pd.to_numeric(df['times'], errors='coerce') >= 0]
        removed = before_len - len(df)
        if removed > 0:
            print(f"[player_time] 删除无效时间段：{removed} 条")
    # 复合键唯一性检查（match_id, times）
    keys = [c for c in ['match_id','times'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[player_time] 删除重复 (match_id, times) 记录：{dup} 条")
    return df

def clean_teamfights(df: pd.DataFrame):
    df = df.copy()  
    # 移除时间区间异常的记录
    if {'start','end'}.issubset(df.columns):
        s = pd.to_numeric(df['start'], errors='coerce')
        e = pd.to_numeric(df['end'], errors='coerce')
        before_len = len(df)
        df = df[(s >= 0) & (e >= 0) & (e > s)]
        removed = before_len - len(df)
        if removed > 0:
            print(f"[teamfights] 删除时间异常记录：{removed} 条")
    # 复合键唯一性检查（match_id, start, end）
    keys = [c for c in ['match_id','start','end'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[teamfights] 删除重复 (match_id, start, end) 记录：{dup} 条")
    return df

def clean_teamfights_players(df: pd.DataFrame):
    df = df.copy()  
    # 填充数值列缺失值为 0
    num_cols = df.select_dtypes(include=[np.number]).columns
    na_total = df[num_cols].isna().sum().sum()
    df[num_cols] = df[num_cols].fillna(0)
    if na_total > 0:
        print(f"[teamfights_players] 填充缺失值：{na_total} 个 -> 0")
    # 复合键唯一性检查（match_id 等多字段）
    keys = [c for c in ['match_id','player_slot','damage','deaths','xp_end','xp_start'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[teamfights_players] 删除重复记录：{dup} 条")
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    return df

def clean_objectives(df: pd.DataFrame):
    df = df.copy()  
    # 统一列名：slot -> player_slot
    if 'slot' in df.columns and 'player_slot' not in df.columns:
        df = df.rename(columns={'slot': 'player_slot'})
    # 统一列名：subtype -> type
    if 'subtype' in df.columns and 'type' not in df.columns:
        df = df.rename(columns={'subtype': 'type'})
    # 移除时间字段异常的记录（time < -120）
    if 'time' in df.columns:
        before_len = len(df)
        t = pd.to_numeric(df['time'], errors='coerce')
        df = df[(t >= -120) | t.isna()].copy()
        removed = before_len - len(df)
        if removed > 0:
            print(f"[objectives] 删除时间异常记录：{removed} 条")
    # 添加阵营列
    if 'team' not in df.columns and 'player_slot' in df.columns:
        df = add_team_from_slot(df, 'player_slot')
    # 复合键唯一性检查
    keys = [c for c in ['match_id','time','type','key','team'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[objectives] 删除重复记录：{dup} 条")
    # 确保存在 type 列
    if 'type' not in df.columns:
        df['type'] = 'unknown'
    return df

def clean_chat(df: pd.DataFrame):
    df = df.copy() 
    # 统一列名：slot -> player_slot
    if 'slot' in df.columns and 'player_slot' not in df.columns:
        df = df.rename(columns={'slot': 'player_slot'})
    # 移除时间字段异常的记录
    if 'time' in df.columns:
        before_len = len(df)
        t = pd.to_numeric(df['time'], errors='coerce')
        df = df[(t >= -120) | t.isna()].copy()
        removed = before_len - len(df)
        if removed > 0:
            print(f"[chat] 删除时间异常记录：{removed} 条")
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    return df

def clean_test_labels(df: pd.DataFrame):
    df = df.copy() 
    # 胜负标签 0/1 转换为布尔值
    if 'radiant_win' in df.columns:
        df['radiant_win'] = boolify01(df['radiant_win'])
    # 主键唯一性检查
    if 'match_id' in df.columns:
        before_len = len(df)
        df = df.drop_duplicates('match_id', keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[test_labels] 删除重复 match_id：{dup} 条")
    return df

def clean_test_player(df: pd.DataFrame, hero_map=None):
    df = df.copy() 
    # 删除缺失关键字段的记录
    need = [c for c in ['match_id','player_slot','hero_id'] if c in df.columns]
    if need:
        before_len = len(df)
        df = df.dropna(subset=need)
        removed = before_len - len(df)
        if removed > 0:
            print(f"[test_player] 删除缺关键列记录：{removed} 条")
    # 复合键唯一性检查（match_id, player_slot）
    keys = [c for c in ['match_id','player_slot'] if c in df.columns]
    if keys:
        before_len = len(df)
        df = df.drop_duplicates(keys, keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[test_player] 删除重复 (match_id, player_slot)：{dup} 条")
    # 合并英雄名称映射
    if hero_map is not None and 'hero_id' in df.columns and 'hero_id_key' in hero_map.columns:
        df['hero_id'] = pd.to_numeric(df['hero_id'], errors='coerce')
        df = df.merge(hero_map[['hero_id_key','hero_name']], left_on='hero_id', right_on='hero_id_key', how='left')
        miss = df['hero_name'].isna().sum()
        if miss > 0:
            print(f"[test_player] 警告：有 {miss} 条 hero_id 未映射到名称")
        df.drop(columns=['hero_id_key'], inplace=True, errors='ignore')
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    return df

def clean_player_ratings(df: pd.DataFrame):
    df = df.copy() 
    # 确保 total_matches 和 total_wins 列为非负
    if {'total_matches','total_wins'}.issubset(df.columns):
        df['total_matches'] = clamp(df['total_matches'], lo=0)
        df['total_wins'] = clamp(df['total_wins'], lo=0)
        bad = df['total_wins'] > df['total_matches']
        if bad.any():
            df.loc[bad, 'total_wins'] = df.loc[bad, 'total_matches']
            print(f"[player_ratings] 发现 wins>matches 的异常 {int(bad.sum())} 条，已更正为 matches 值")
    # 将 TrueSkill 指标裁剪为非负
    for c in ['trueskill_mu','trueskill_sigma']:
        if c in df.columns:
            df[c] = clamp(df[c], lo=0)
    # 用户唯一性检查
    if 'account_id' in df.columns:
        before_len = len(df)
        df = df.drop_duplicates('account_id', keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[player_ratings] 删除重复 account_id：{dup} 条")
    return df

def clean_match_outcomes(df: pd.DataFrame):
    df = df.copy() 
    # 确保 win 和 rad 列为整数类型
    if 'win' in df.columns:
        df['win'] = pd.to_numeric(df['win'], errors='coerce').fillna(0).astype(int)
    if 'rad' in df.columns:
        df['rad'] = pd.to_numeric(df['rad'], errors='coerce').fillna(0).astype(int)
    # 主键唯一性检查
    if 'match_id' in df.columns:
        before_len = len(df)
        df = df.drop_duplicates('match_id', keep='first')
        dup = before_len - len(df)
        if dup > 0:
            print(f"[match_outcomes] 删除重复 match_id：{dup} 条")
    return df

def clean_purchase_log(df: pd.DataFrame, item_df=None):
    df = df.copy() 
    # 移除异常时间记录，并添加发生分钟列
    if 'time' in df.columns:
        before_len = len(df)
        t = pd.to_numeric(df['time'], errors='coerce')
        df = df[(t >= -120) | t.isna()].copy()
        removed = before_len - len(df)
        if removed > 0:
            print(f"[purchase_log] 删除时间异常记录：{removed} 条")
        df['minute'] = (t // 60).astype('Int64')
    # 合并物品名称映射
    if item_df is not None and {'item_id'}.issubset(df.columns):
        df = df.merge(item_df[['item_id','item_name']], on='item_id', how='left')
        miss = df['item_name'].isna().sum()
        if miss > 0:
            print(f"[purchase_log] 警告：有 {miss} 条 item_id 未映射到名称")
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    return df

def clean_ability_upgrades(df: pd.DataFrame, ability_df=None):
    df = df.copy() 
    # 统一列名：ability -> ability_id
    if 'ability' in df.columns and 'ability_id' not in df.columns:
        df = df.rename(columns={'ability': 'ability_id'})
    # 限制 level 最小值为 1
    if 'level' in df.columns:
        df['level'] = clamp(df['level'], lo=1)
    # 移除异常时间记录
    if 'time' in df.columns:
        before_len = len(df)
        t = pd.to_numeric(df['time'], errors='coerce')
        df = df[(t >= -120) | t.isna()].copy()
        removed = before_len - len(df)
        if removed > 0:
            print(f"[ability_upgrades] 删除时间异常记录：{removed} 条")
    # 合并技能名称映射
    if ability_df is not None and {'ability_id'}.issubset(df.columns):
        df = df.merge(ability_df[['ability_id','ability_name']], on='ability_id', how='left')
        miss = df['ability_name'].isna().sum()
        if miss > 0:
            print(f"[ability_upgrades] 警告：有 {miss} 条 ability_id 未映射到名称")
    # 添加阵营列
    df = add_team_from_slot(df, 'player_slot')
    return df


## 读取原始数据

尝试读取所有原始的比赛数据 CSV 文件，并将其加载为 DataFrame：

- `matches_df`：比赛场次信息  
- `players_df`：比赛玩家统计  
- `player_time_df`：玩家随时间的事件数据  
- `teamfights_df`：团队战斗事件  
- `teamfights_players_df`：团队战斗中玩家的事件数据  
- `objectives_df`：比赛目标（建筑物、Roshan 等）事件  
- `chat_df`：比赛内的聊天记录  
- `test_labels_df`：测试集比赛结果标签  
- `test_player_df`：测试集比赛玩家数据  
- `player_ratings_df`：玩家综合评分数据  
- `match_outcomes_df`：比赛结果汇总数据  
- `purchase_log_df`：物品购买日志  
- `ability_upgrades_df`：英雄技能升级日志表

（上述文件如存在则会被读取；未提供的文件将得到 `None`，在后续清洗时将被跳过。）


In [5]:
# ====== 读取原始数据 ======
matches_df            = read_csv_if_exists('match.csv')             # 比赛场次主表
players_df            = read_csv_if_exists('players.csv')           # 玩家数据表
player_time_df        = read_csv_if_exists('player_time.csv')       # 玩家时间序列数据表
teamfights_df         = read_csv_if_exists('teamfights.csv')        # 团队战斗事件表
teamfights_players_df = read_csv_if_exists('teamfights_players.csv')# 团队战斗玩家事件表
objectives_df         = read_csv_if_exists('objectives.csv')        # 目标/建筑物事件表
chat_df               = read_csv_if_exists('chat.csv')              # 聊天记录表
test_labels_df        = read_csv_if_exists('test_labels.csv')       # 测试集比赛标签表
test_player_df        = read_csv_if_exists('test_player.csv')       # 测试集比赛玩家表
player_ratings_df     = read_csv_if_exists('player_ratings.csv')    # 玩家评分表
match_outcomes_df     = read_csv_if_exists('match_outcomes.csv')    # 比赛结果汇总表
purchase_log_df       = read_csv_if_exists('purchase_log.csv')      # 物品购买日志表
ability_upgrades_df   = read_csv_if_exists('ability_upgrades.csv')  # 英雄技能升级日志表

# 如未读取到任何数据文件，则提示检查路径
if all(df is None for df in [matches_df, players_df, player_time_df, teamfights_df, teamfights_players_df,
                             objectives_df, chat_df, test_labels_df, test_player_df, player_ratings_df,
                             match_outcomes_df, purchase_log_df, ability_upgrades_df]):
    print("[INFO] 没有读取到任何数据文件，请检查 DATA_DIR 路径或原始文件是否存在。")


[READ] match.csv: shape=(50000, 13)
[READ] players.csv: shape=(500000, 73)
[READ] player_time.csv: shape=(2209778, 32)
[READ] teamfights.csv: shape=(539047, 5)
[READ] teamfights_players.csv: shape=(5390470, 8)
[READ] objectives.csv: shape=(1173396, 9)
[READ] chat.csv: shape=(1439488, 5)
[READ] test_labels.csv: shape=(100000, 2)
[READ] test_player.csv: shape=(1000000, 4)
[READ] player_ratings.csv: shape=(834226, 5)
[READ] match_outcomes.csv: shape=(1828588, 10)
[READ] purchase_log.csv: shape=(18193745, 4)
[READ] ability_upgrades.csv: shape=(8939599, 5)


## 清洗并保存主数据表

对已成功加载的每张原始数据表，调用上面定义的清洗函数进行处理，并将清洗后的数据保存为 CSV 文件（文件名附加 `_clean` 后缀）。在此过程中收集各表清洗前后的行数统计，最终汇总输出清洗结果摘要，便于对比清洗前后的数据规模变化。


In [6]:
# ====== 清洗流水线 ======
summary = []
# 逐表执行清洗并保存结果
if matches_df is not None:
    m_clean = clean_matches(matches_df, cluster_df, patch_df); save_df(m_clean, 'match')
    summary.append(['match', len(matches_df) if matches_df is not None else 0, len(m_clean) if m_clean is not None else 0])
if players_df is not None:
    p_clean = clean_players(players_df, hero_map); save_df(p_clean, 'players')
    summary.append(['players', len(players_df), len(p_clean)])
if player_time_df is not None:
    pt_clean = clean_player_time(player_time_df); save_df(pt_clean, 'player_time')
    summary.append(['player_time', len(player_time_df), len(pt_clean)])
if teamfights_df is not None:
    tf_clean = clean_teamfights(teamfights_df); save_df(tf_clean, 'teamfights')
    summary.append(['teamfights', len(teamfights_df), len(tf_clean)])
if teamfights_players_df is not None:
    tfp_clean = clean_teamfights_players(teamfights_players_df); save_df(tfp_clean, 'teamfights_players')
    summary.append(['teamfights_players', len(teamfights_players_df), len(tfp_clean)])
if objectives_df is not None:
    obj_clean = clean_objectives(objectives_df); save_df(obj_clean, 'objectives')
    summary.append(['objectives', len(objectives_df), len(obj_clean)])
if chat_df is not None:
    chat_clean = clean_chat(chat_df); save_df(chat_clean, 'chat')
    summary.append(['chat', len(chat_df), len(chat_clean)])

print("\n=== 清洗摘要 ===")
summary_df = pd.DataFrame(summary, columns=['table','raw_rows','clean_rows']) if summary else pd.DataFrame()
display(summary_df)


[SAVE] cleaned\match_clean.csv -> shape=(50000, 18)
[players] 填充缺失值：11190369 个 -> 0
[players] 警告：有 37 条 hero_id 未映射到名称（可能是新英雄/映射表需更新）
[SAVE] cleaned\players_clean.csv -> shape=(500000, 76)
[SAVE] cleaned\player_time_clean.csv -> shape=(2209778, 32)
[teamfights] 删除时间异常记录：1279 条
[SAVE] cleaned\teamfights_clean.csv -> shape=(537768, 5)
[teamfights_players] 删除重复记录：15211 条
[SAVE] cleaned\teamfights_players_clean.csv -> shape=(5375259, 9)
[objectives] 删除重复记录：1277 条
[SAVE] cleaned\objectives_clean.csv -> shape=(1172119, 9)
[chat] 删除时间异常记录：2925 条
[SAVE] cleaned\chat_clean.csv -> shape=(1436563, 6)

=== 清洗摘要 ===


,table,raw_rows,clean_rows
0,match,50000,50000
1,players,500000,500000
2,player_time,2209778,2209778
3,teamfights,539047,537768
4,teamfights_players,5390470,5375259
5,objectives,1173396,1172119
6,chat,1439488,1436563


## 清洗附加数据表

对测试集和其他附加数据表进行清洗，与上述流程类似，并输出相应的摘要表：

该步骤会清洗 **test_labels**、**test_player**、**player_ratings**、**match_outcomes**、**purchase_log**、**ability_upgrades** 等表，并将清洗结果保存。同时收集这些表清洗前后的行数并输出第二个清洗摘要表用于展示。


In [7]:
# ====== 清洗附加数据 ======
summary = []
# 清洗测试集及其他附加表并保存结果
if test_labels_df is not None:
    tl_clean = clean_test_labels(test_labels_df); save_df(tl_clean, 'test_labels')
    summary.append(['test_labels', len(test_labels_df), len(tl_clean)])
if test_player_df is not None:
    tp_clean = clean_test_player(test_player_df, hero_map); save_df(tp_clean, 'test_player')
    summary.append(['test_player', len(test_player_df), len(tp_clean)])
if player_ratings_df is not None:
    pr_clean = clean_player_ratings(player_ratings_df); save_df(pr_clean, 'player_ratings')
    summary.append(['player_ratings', len(player_ratings_df), len(pr_clean)])
if match_outcomes_df is not None:
    mo_clean = clean_match_outcomes(match_outcomes_df); save_df(mo_clean, 'match_outcomes')
    summary.append(['match_outcomes', len(match_outcomes_df), len(mo_clean)])
if purchase_log_df is not None:
    pl_clean = clean_purchase_log(purchase_log_df, item_df); save_df(pl_clean, 'purchase_log')
    summary.append(['purchase_log', len(purchase_log_df), len(pl_clean)])
if ability_upgrades_df is not None:
    au_clean = clean_ability_upgrades(ability_upgrades_df, ability_df); save_df(au_clean, 'ability_upgrades')
    summary.append(['ability_upgrades', len(ability_upgrades_df), len(au_clean)])

print("\n=== 清洗摘要 ===")
summary_df = pd.DataFrame(summary, columns=['table','raw_rows','clean_rows']) if summary else pd.DataFrame()
display(summary_df)


[SAVE] cleaned\test_labels_clean.csv -> shape=(100000, 2)
[test_player] 警告：有 15 条 hero_id 未映射到名称
[SAVE] cleaned\test_player_clean.csv -> shape=(1000000, 6)
[SAVE] cleaned\player_ratings_clean.csv -> shape=(834226, 5)
[match_outcomes] 删除重复 match_id：914294 条
[SAVE] cleaned\match_outcomes_clean.csv -> shape=(914294, 10)
[SAVE] cleaned\purchase_log_clean.csv -> shape=(18193745, 7)
[SAVE] cleaned\ability_upgrades_clean.csv -> shape=(8939599, 7)

=== 清洗摘要 ===


,table,raw_rows,clean_rows
0,test_labels,100000,100000
1,test_player,1000000,1000000
2,player_ratings,834226,834226
3,match_outcomes,1828588,914294
4,purchase_log,18193745,18193745
5,ability_upgrades,8939599,8939599


## 清洗结果抽样预览

从部分清洗后的输出文件中抽取前几行进行展示，以快速检查清洗后的数据格式和内容是否符合预期。


In [8]:
# ====== 清洗结果预览 ======
for name in ['match_clean.csv', 'players_clean.csv', 'objectives_clean.csv', 'purchase_log_clean.csv']:
    p = OUT_DIR / name
    if p.exists():
        print(f"\n>>> {p}")
        display(pd.read_csv(p, nrows=5))



>>> cleaned\match_clean.csv


,match_id,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster,start_time_dt,long_match,region,patch,patch_date
0,0,1446750112,2375,1982,4,3,63,1,22,True,0,1,155,2015-11-05 19:01:52,False,SINGAPORE,6.85,2015-09-24 20:00:00
1,1,1446753078,2582,0,1846,63,0,221,22,False,0,2,154,2015-11-05 19:51:18,False,SINGAPORE,6.85,2015-09-24 20:00:00
2,2,1446764586,2716,256,1972,63,48,190,22,False,0,0,132,2015-11-05 23:03:06,False,EUROPE,6.85,2015-09-24 20:00:00
3,3,1446765723,3085,4,1924,51,3,40,22,False,0,0,191,2015-11-05 23:22:03,False,AUSTRIA,6.85,2015-09-24 20:00:00
4,4,1446796385,1887,2047,0,0,63,58,22,True,0,0,156,2015-11-06 07:53:05,False,SINGAPORE,6.85,2015-09-24 20:00:00



>>> cleaned\players_clean.csv


,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue,team,kills_outlier,hero_name
0,0,0,86,0,3261,10960,347,362,9,3,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,Radiant,False,Rubick
1,0,1,51,1,2954,17760,494,659,13,3,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,Radiant,False,Clockwerk
2,0,0,83,2,110,12195,350,385,0,4,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,Radiant,False,Treant Protector
3,0,2,11,3,1179,22505,599,605,8,4,...,13.0,0.0,0.0,0.0,0.0,0.0,0.0,Radiant,False,Shadow Fiend
4,0,3,67,4,3307,23825,613,762,20,3,...,23.0,0.0,0.0,0.0,0.0,0.0,0.0,Radiant,False,Spectre



>>> cleaned\objectives_clean.csv


,match_id,key,player1,player2,player_slot,type,team,time,value
0,0,NaN,0,6,0.0,CHAT_MESSAGE_FIRSTBLOOD,NaN,1,309
1,0,NaN,3,-1,3.0,CHAT_MESSAGE_TOWER_KILL,2.0,894,2
2,0,NaN,2,-1,NaN,CHAT_MESSAGE_ROSHAN_KILL,2.0,925,200
3,0,NaN,1,-1,1.0,CHAT_MESSAGE_AEGIS,NaN,925,0
4,0,NaN,7,-1,7.0,CHAT_MESSAGE_TOWER_KILL,3.0,1016,3



>>> cleaned\purchase_log_clean.csv


,item_id,time,player_slot,match_id,minute,item_name,team
0,44,-81,0,0,-2,tango,Radiant
1,29,-63,0,0,-2,boots,Radiant
2,43,6,0,0,0,ward_sentry,Radiant
3,84,182,0,0,3,flying_courier,Radiant
4,46,197,0,0,3,tpscroll,Radiant


## 运行环境与依赖

- Python ≥ 3.9  
- 必需：`pandas`, `numpy`  
- 可选：`jupyter`（若在 Notebook 中运行）、`pyarrow`（读写加速）

**示例安装：**
```bash
pip install -U pandas numpy jupyter
# 如需快速打开本文件：
jupyter notebook



## 本地运行步骤
1. 将原始 CSV 数据集（例如 Kaggle Dota 2 Matches）放置到 `DATA_DIR` 指定的目录下（默认路径为 `/workspace/Dota2data`）。  
2. 打开本 Notebook；如果数据路径与默认不同，请在第一个代码单元格中修改 `DATA_DIR` 参数。  
3. 按顺序运行所有单元格。清洗完成后，输出的 CSV 文件将在 `OUT_DIR`（默认 `/workspace/cleaned`）目录下生成，文件名格式为 `*_clean.csv`。  
4. 如出现 **hero_id / item_id / ability_id 无法映射** 的警告，请更新对应的映射表文件（`hero_names.csv`、`item_ids.csv`、`ability_ids.csv`）后重新运行清洗流程。  
5. 可以将本 Notebook 中定义的清洗函数迁移或复用到独立的脚本（如 `cleaner.py`），并加入命令行参数、日志和分批/并行处理等功能，以适应更大规模的数据清洗任务。
